# PromptTemplate

PromptTemplate is used to generate dynamic prompts by injecting variables at runtime and creating parameterized prompts.

Bodhilib supports 2 ways of injecting variables using PromptTemplate -

1. **fstring**

    [fstring](https://peps.python.org/pep-0498/) uses the python's native string interpolation features to inject named variables. 
    
    In `fstring`, you define your named variable using single curly braces, `{` and `}`, and during variable injection, the value is replaced by the variable value passed at runtime.
    
    For e.g. for a fstring template - "The {movie} is the best super-hero movie ever.". At runtime, the `{movie}` is going to be replaced by the value of variable `movie` and produce a dynamic string to be used as prompt.

2. **jinja2**

    Though powerful, `fstring` is limited in its features. It cannot have `for-loops`, `if-else conditionals` etc. To overcome these limitations, bodhilib also supports [jinja2](https://palletsprojects.com/p/jinja/) as a templating engine.

    In `jinja2` format, you define your prompt template using the [jinja2 format](https://palletsprojects.com/p/jinja/), and then these templates are safely injected with variables at runtime to produce the dynamic prompt.


While creating the PromptTemplate, you specify which format you want to use using the `format` parameter. Then PromptTemplate uses that templating engine to inject variables and generate the Prompt.

Let's try it out.

In [1]:
# import PromptTemplate class

from bodhilib import PromptTemplate

In [2]:
# create the template using fstring format

template = PromptTemplate(
    template="The {movie} is the best super-hero movie of all times. Do you agree?",
    format="fstring",
)

In [3]:
options = ["Avengers: End Game", "Justice League", "The Dark Knight"]

# Use the PromptTemplate `to_prompts` method to generate the prompts
for option in options:
    print(">", template.to_prompts(movie=option))

> [Prompt(text='The Avengers: End Game is the best super-hero movie of all times. Do you agree?', role=<Role.USER: 'user'>, source=<Source.INPUT: 'input'>)]
> [Prompt(text='The Justice League is the best super-hero movie of all times. Do you agree?', role=<Role.USER: 'user'>, source=<Source.INPUT: 'input'>)]
> [Prompt(text='The The Dark Knight is the best super-hero movie of all times. Do you agree?', role=<Role.USER: 'user'>, source=<Source.INPUT: 'input'>)]


Let's pass these prompts to llm and see what it thinks:

In [4]:
# setup the environment variables
# input your OpenAI API key when prompted

import os
from getpass import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your API key: ')

In [5]:
# Get instance of OpenAI Chat LLM service
from bodhilib import get_llm

llm = get_llm("openai_chat", model="gpt-3.5-turbo")

In [6]:
import textwrap

# Use the PromptTemplate `to_prompts` method to generate the prompts
for option in options:
    prompts = template.to_prompts(movie=option)
    response = llm.generate(prompts)
    print(f"> {prompts[0].text}\n")
    print(textwrap.fill(response.text, 100))
    print("---")

> The Avengers: End Game is the best super-hero movie of all times. Do you agree?

As an AI language model, I don't have personal opinions. However, "Avengers: Endgame" is often considered one of the
best superhero movies based on popular opinion and box office success. It received positive reviews and achieved great
commercial success, breaking numerous records at the time of its release. However, the notion of the "best" superhero
movie can differ depending on individual preferences and subjective opinions. There are many other superhero movies that
have also garnered critical acclaim and have a dedicated fan base. Ultimately, the best superhero movie is a matter of
personal taste.
---
> The Justice League is the best super-hero movie of all times. Do you agree?

As an AI, I don't have personal opinions or preferences. However, opinions on the best superhero movie can vary among
individuals. The Justice League movie has its fans and critics, and different people have different prefer

---

🎉 We just generated response from LLM using our dynamically generated prompts using PromptTemplate.

Next, let's look at generating a more complex prompt that involves looping through options to generate a Prompt. We will use `jinja2` format to create this complex prompt.

In [7]:
template = """
Which is the best super-hero movie of all times from among the options below? and why?

{% for option in options -%}
{{ loop.index }}. {{ option }}
{% endfor -%}
"""
template = PromptTemplate(template=template, format="jinja2")

In [8]:
prompts = template.to_prompts(options=options)
print(prompts[0].text)


Which is the best super-hero movie of all times from among the options below? and why?

1. Avengers: End Game
2. Justice League
3. The Dark Knight



Let's see what our LLM model thinks about the above. Let's pass the prompts to LLM service and get the response.

In [9]:
import textwrap

response = llm.generate(prompts)
print(textwrap.fill(response.text, 100))

As an AI language model, I don't have personal opinions. However, based on critical reception and popular opinion, "The
Dark Knight" is often regarded as one of the best superhero movies of all time. It is praised for its dark and gritty
tone, compelling storyline, exceptional performances (including Heath Ledger's portrayal of the Joker), and its
exploration of complex themes. "The Dark Knight" has received widespread acclaim for its direction, writing, and
cinematography, making it a standout among superhero films.


---

🎉 Yay! we have our our first response from LLM using a complex dynamic prompt generated using `jinja2` template.

Next, we look at `PromptSource` and finding the right prompt for our use case.